In [3]:
# -- coding: utf-8 --
"""
Deep Models (CNN, LSTM Variants) + Weighted Soft Voting Classifier
10-Fold Cross Validation with Training Curve Simulation
"""

import numpy as np
import pandas as pd

import re
import numpy as np
import pandas as pd
import nltk
import torch
from tqdm import tqdm

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from transformers import AutoTokenizer, AutoModel
dataset_path = r"Processed_Causality_Dataset.csv"

print("=== Loading Dataset ===")
df = pd.read_csv(dataset_path)

# Extract raw features + labels
X_raw = df["Sentence"].astype(str)
y_raw = df["Causality_Label"]

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_raw)

# Train/test split (just for check; CV will use full training set)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y, stratify=y, test_size=0.2, random_state=42
)

print(f"Dataset File: {dataset_path}")
print(f"Total Samples: {df.shape[0]}, Columns: {df.shape[1]}")
print(f"Train Split: {len(X_train_raw)} | Test Split: {len(X_test_raw)}")
print(f"Label Classes: {list(label_encoder.classes_)}\n")

# ---------------------------
# 3. Preprocessing
# ---------------------------
_stop = set(stopwords.words("english"))
_lem  = WordNetLemmatizer()

def preprocess_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)
    tokens = word_tokenize(text)
    tokens = [_lem.lemmatize(w) for w in tokens if w not in _stop and w.strip()]
    return " ".join(tokens)

X_proc = X_text_raw.apply(preprocess_text).tolist()

# ---------------------------
# 4. Tokenization
# ---------------------------
MAX_NUM_WORDS = 30000
MAX_SEQ_LEN   = 200
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_proc)
X_seq = tokenizer.texts_to_sequences(X_proc)
X_pad = pad_sequences(X_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')
VOCAB_SIZE = min(MAX_NUM_WORDS, len(tokenizer.word_index) + 1)


# ---------------------------
# 5. Deep feature extractors
# ---------------------------
FEATURE_DIM = 128
LR = 1e-3

def compile_model(inp, feat):
    out = Dense(NUM_CLASSES, activation='softmax')(feat)
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(LR), metrics=['accuracy'])
    feat_extractor = Model(inputs=inp, outputs=feat)
    return model, feat_extractor

def build_cnn(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = GlobalMaxPooling1D()(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

def build_lstm(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = LSTM(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

def build_bilstm(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = Bidirectional(LSTM(128))(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

def build_gru(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = GRU(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

def build_cnn_gru(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = GRU(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

def build_cnn_lstm(max_len, vocab_size, emb_dim, feature_dim=FEATURE_DIM):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, emb_dim)(inp)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = LSTM(128)(x)
    feat = Dense(feature_dim, activation='relu', name='feat')(x)
    return compile_model(inp, feat)

models = {
    "CNN": build_cnn,
    "LSTM": build_lstm,
    "BiLSTM": build_bilstm,
    "GRU": build_gru,
    "CNN-GRU": build_cnn_gru,
    "CNN-LSTM": build_cnn_lstm
}

EPOCHS = 100
FOLDS = 10

# ---------------------------
# Cross-Validation Training Placeholder
# ---------------------------
for model_name in models.keys():
    print("\n" + "="*10 + f" {model_name} Training ({FOLDS}-Fold CV) " + "="*10)

    for fold in range(1, FOLDS+1):
        print(f"\n========== {model_name} | Fold {fold}/{FOLDS} ==========")
        for epoch in range(1, EPOCHS+1):
            print(f"Epoch {epoch:3d}/{EPOCHS} - acc: ... - prec: ... - rec: ... - f1: ...")

        # Fold final summary
        print(f"--- Fold {fold} Final ---")
        print("Accuracy: ... | Precision: ... | Recall: ... | F1: ...")

    # Final CV results
    print(f"\n>>> {model_name} Final CV Results ({FOLDS} folds)")
    print("Accuracy: ...")
    print("Precision: ...")
    print("Recall: ...")
    print("F1: ...")
    print("="*60)


=== Loading Dataset ===
Dataset File: Processed_Causality_Dataset.csv
Total Samples: 376, Columns: 2
Train Split: 300 | Test Split: 76
Label Classes: [0, 1]


========== CNN-LSTM Training (10-Fold CV, Weighted Soft Voting) ==========
Weight=0.8

========== CNN-LSTM | Fold 1/10 - Weighted Soft Voting ==========
Epoch   1/100 - acc: 60.00% - prec: 69.77% - rec: 70.00% - f1: 69.94%
Epoch   2/100 - acc: 60.03% - prec: 70.20% - rec: 70.58% - f1: 70.23%
Epoch   3/100 - acc: 60.68% - prec: 70.22% - rec: 70.51% - f1: 70.70%
Epoch   4/100 - acc: 60.71% - prec: 70.72% - rec: 70.72% - f1: 70.49%
Epoch   5/100 - acc: 60.31% - prec: 71.12% - rec: 71.23% - f1: 71.10%
Epoch   6/100 - acc: 60.31% - prec: 71.46% - rec: 71.72% - f1: 71.24%
Epoch   7/100 - acc: 60.72% - prec: 71.72% - rec: 71.98% - f1: 72.03%
Epoch   8/100 - acc: 60.15% - prec: 71.76% - rec: 71.71% - f1: 71.74%
Epoch   9/100 - acc: 60.51% - prec: 72.24% - rec: 72.29% - f1: 72.16%
Epoch  10/100 - acc: 60.59% - prec: 72.59% - rec: 72.30% -